# 01 - Global and Local Scopes

We know that our variables are stored in different scopes which means that we can have seemingly duplicate assignments to the same variable. But where does this binding exist? It exists in a **namespace**. 

A **namespace** can be simply thought of as a table that contains the label and the memory reference that it's pointing to. Every scope has a namespace.

The **global** scope is essentially the **module** scope. They're synonymous. It spans a **single file only**. There is no truly global scope (across all the modules in our entile app), but there is one exception: **built-in objects** like `True`, `False`, `None`, `dict`, `print`. All of these are living in a **namespace** somewhere.

If you reference a variable inside a scope and Python doesn't find it in that **namespace**, it will look for it in an **enclosing* scope's namespace**. So, in the example below, it will look for it in the Built-in scope.

<img src=s7-images/7.1.png width=450/>

If we create a module called **module1.py** with 1 line of code: `print(a)`, then Python looks within **module1.py**'s namespace for `print` and `a`. It finds neither of them. So, it looks in one scope higher and finds a `print` object in a higher scope but not `a`, so we get a runtime error. If we define a new `print` variable set to e.g. a `lambda` expression, then Python will look and use that definition as opposed to the built-in `print` function because of scope hierarchy. This is known as **masking** and it's something we don't want to really do.


**Local Scope**

Variable bindings defined within a function are not created until the function is called. Everytime a function is called **a new scope is created**; this makes sense because we can call a function with different values, so variables are able to have different bindings with each call.

During compile time (not runtime), Python looks at a line like: `def my_func(a, b)` and sees that `a` and `b` are going to be **local** to `my_func`. **But it doesn't create the namespaces or scope yet - that's done at runtime when the function is called - all it does it make a determination of the labels that are within that function and marks them as either local or global or something else.

Once we finally call the function, that's when the namespace containing all the assignments of variables to their values are binded. Once the function exits, **the scope is deleted, because it's no longer needed**.

Let's take a look at a quick example:

<img src=s7-images/7.2.png width=500/>

The key thing to realise from the image is that, because Python cannot find `a` within `my_func`'s local scope, it goes one scope out to the **global/module** scope and finds it there. Also note that the variable `my_func` is created within the global scope but only as a pointer to a function object. The function isn't called yet of course.

**Reference Counting**

Once we run `my_func(b=300)`, the `300` integer object gets created (if it doesn't already exist) and it's reference count increments by 1. Once `my_func` finishes run, the **scope is destroyed** and the reference count for the **integer object** is decremented by 1. Thereafter, that binding of `b` **goes out of scope**.

Above we talked about **accessing** the value `a` from within the local scope where it doesn't exist locally - and it does so by scope hierarchy. What about **modifying** `a`?

<img src=s7-images/7.3.png width=400/>


**Encountering a function definiting at compile time**

Let's emphasise exactly what Python does at compile time. It will **scan** for any labels/variables (LHS of an assignment) anywhere within that function and will declare: If they haven't been specified as **global**, then I will specify them as **local**. Variables, namespaces and scopes are **NOT** created yet. 

Also note: **variables referenced but not assigned a value anywhere in that function will not be local** and Python will, **at runtime**, look for them in any scope outside of the local scope. So, if you were to print `True` within a function, since we haven't assigned it, e.g. `True = 10`, Python looks elsewhere for it at runtime.

So, to summarise:
- As soon as you **assign** a variable, and not **reference**, it becomes local, unless the `global` keyword is used.
- Compile time is for specifying whether variables/labels within a function are **local** or **global**.
- Runtime is for creating the namespace and performing assignment/binding of variables to their objects in memory. Note that parameters to functions are local.

Let's look at a tricky example:

In [131]:
a = 10
def func_1():
    print(a)
    a = 100

func_1() # RUNTIME ERROR

UnboundLocalError: local variable 'a' referenced before assignment

What's happening? 

- At compile time, since `a` isn't declared `global` anywhere within the function, it defaults to local, so Python has a label of `a` waiting to be assigned.
- At runtime, we may be tempted to say that `a` doesn't exist so look in the global scope. But that's false. `a` does exist! At that point, the **namespace** has been created and `a` exists there as a local variable with no value assigned.
- We then try to reference it but a value has been assigned, so weget a runtime error.

# 02 - Nonlocal Scopes

Nonlocal scopes are created when we nest a function definition within a function. From within `inner_func`, we can access its local scope, the global scope but also inner scope's enclosing scope. This is scope of `outer_func` and it is called the **nonlocal scope**.

<img src=s7-images/7.4.png width=500/>

Below is a basic example:

In [132]:
def outer_func():
    a = 10
    
    def inner_func():
        print(a)
    
    inner_func()

outer_func()

10


When `inner_func` is reached during compile, it sees mention of `a` but no assignment, therefore it specifies `a` as something other than local. During runtime, when we reach `print(a)`, Python has to go outwards in scope to look for `a`. It finds it in the nonlocal scope. If it weren't there, it would go and look for it in the global scope.

If we have a variable `b` defined in the global scope and then we create an outer and inner function where the inner function contains `global b`, Python will jump straigh to the most outer scope, which we know as the global scope. If we follow `global b` by something like `b=100`, then `b` will be globally changed in the module. 

Modifying nonlocal variables is fairly intuitive:

<img src=s7-images/7.5.png width=500/>

What if we want to modify a **nonlocal** variable from within a local scope? We use the `nonlocal` keyword. It is **important** to note that, whenever Python is told that a variable is nonlocal, it will look for it in the enclosing local scopes chain until it **first** encounters the specified variable name. **It will never look in the global scope**. If we want it to find the global variable, we must use the `global` keyword.

In [133]:
def outer_func():
    x = 'hello'
    
    def inner_func():
        nonlocal x
        x = 'python'
    
    inner_func()
    print(x)

outer_func()

python


To emphasise once more, if we are within a twice-nested (inner2) scope and we assert `nonlocal` onto a variable called `c` which exists in the enclosing, once-nested (inner1) scope and also in the outer scope - then, Python works its way from most inner to most outer. Once it finds the first mention of an object with the same name, that object's reference count gets incremented by 1.

<img src=s7-images/7.6.png width=500/>

# 03 - Closures

Consider the following:

In [134]:
def outer():
    ##CLOSURE START##
    x = 'python'
    def inner():
        print(x)      
    ##CLOSURE END##
    inner()
    
outer()

python


First note that `inner()` is not created until `outer()` is called. Now, we know that the `x` within `inner()` refers to the one in `outer's scope`. Therefore, this **nonlocal** variable is a **free variable** in `inner()`.

When we consider `inner()` we really must also consider `x='python'` because the two live and are bound together. This is called a **closure**. We also say `inner()` encloses its free variable.

Now what happens if we return `inner()` instead of calling it inside of `outer()`?

In [135]:
def outer():
    ##CLOSURE START##
    x = 'python'
    def inner():
        print(x)
    ##CLOSURE END##
    
    return inner

When we return `inner`, we are actually **returning a closure**. Because, we are returning the function **AND** its free variable. Remember `x` is not defined within `inner()`, yet we still have access to `x` from returning `inner` because the two are bound together in a **closure**. By assigning `outer()` (not `outer`), which returns a closure, to a variable `fn`, we can say that `fn` is now that closure.

In [136]:
fn = outer()

`outer` runs when we execute the above. In the Subsections prior, we said that when a function finishes running, its scope is destroyed. So, we would expect `x='python'` to vanish too. If this is true, then executing `fn()`, which is *like* executing `inner()`, should produce a runtime error. But it doesn't, and that's because the **closure** captured the variable `x` so it still has access to it.

In [137]:
fn()

python


We also say that `x` is a multi-scoped variable (shared between `outer` and the closure). This means the label is in two scopes but they **always reference the same value**. So how does Python do this?

It creates a **cell** as an intermediary object. This is a literal object with a memory address just like e.g. integer objects.

<img src=s7-images/7.7.png width=500 />

If we didn't have these nested functions and `x` was defined in a purely local scope only, then we would expect `x` to point to a string object. But once Python sees that we have a shared variable between multiple scopes, it sets up this cell. Then tells `x` in the outer scope (x.outer; not actual object, just notation) and `x` in the inner scope to both point towards that cell.

This is infact exactly how nonlocal variables work - it's got nothing to do with closures - it's just that you have a variable in two different scopes. Now we can understand from the last subsection that we are always referring to the same string object. But we chain via a cell as opposed to looking for the same label in an outer scope which looks for another label in a further outer scope until eventually we reach the memory address containing the object. This description would be like saying `inner.x -> outer.x -> string object` in contrast to the image above.

If we wanted to change the string from `'python'` to e.g. `'hello'` from within an inner scope, we would first use the keyword `nonlocal`, which creates the **closure**, and then make the change. Python would then make the **cell** point to a new string object.

Going back to the `fn()` line, Python does indeed destroy `outer.x` but `inner.x` and the cell object remain. So, we can access the string via this route. All good!

**Thinking about closures practically**

It will often be sufficient to just think of closures as ***function*** plus an ***extended scope*** which contains all the free variables.

Let's make sure we understand all aspects of the following:

<img src=s7-images/7.8.png width=450 />

- `a = 100` is a local variable.
- `x = 'python'` will be a local variable until ` def inner():` runs, at which point Python will understand that, since `inner` contains a reference to `x`, a closure will need to be generated, i.e. `inner` is made into a **closure**. This involves creating a cell where the `x` in `outer` and the `x` reference in `inner` both point to it, 
- `a = 10` is an assignment and contains no `nonlocal/global` statement, therefore, its local to `inner()`.

**I want to emphasise that the closure is created when the function is created, not when the function is called**.

We can verify these statements using introspection:

In [138]:
print(fn.__code__.co_freevars) #tuple of all the free variables of fn
print(fn.__closure__) #tuple describing the cell memory location and what it points to

('x',)
(<cell at 0x7fb4f00fb0d0: str object at 0x7fb4f63bdd70>,)


**Multiple instances of closures**

Every time we run a function, a new scope is created. If that function generates a closure, a new closure is created every time as well. These closures do not have the same extended scope - mutating free variables in one extended scope has no impact on another extended scope.

In [139]:
def counter():
    count = 0 # local variable
    
    def inc():
        nonlocal count  # this is the count variable in counter
        count += 1
        return count
    return inc

f1 = counter()
f2 = counter()
f1.__closure__, f2.__closure__

((<cell at 0x7fb4f00f97b0: int object at 0x7fb4f77180d0>,),
 (<cell at 0x7fb4f00f8f70: int object at 0x7fb4f77180d0>,))

In [140]:
print(f1(), f1(), f1(), f1())
print(f2()) # count in f1 closure UNAFFECTED

1 2 3 4
1


<a id='shared_extended_scope_example'></a>

#### **Shared Extended Scopes**

The following process allows us to have 3 branches to a cell that points to an object, as opposed to the 2 branches last time (x.outer and x.inner). We can set up nonlocal variables in different inner functions that reference the same outer scope variable, i.e. we have a free variable that is shared between two closures.

In [141]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    
    return inc1, inc2

f1, f2 = outer() # basically f1 = inc1 and f2 = inc2

print(f1(), f1(), f1(), f1())
print(f2()) # count in f1 closure AFFECTED

1 2 3 4
5


**Shared Extended Scopes - Subtle Bug**

Let's say we want to (correctly) create multiple closures without a shared scope.

In [142]:
def adder(n):
    def inner(x):
        return x + n
    return inner

add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

But, let's replicate this with a loop. We may think of using a lambda expression like the following:

In [143]:
def create_adders():
    adders = []
    
    for n in range(1, 5): # n here is a local variable of 'create_adders'
        adders.append(lambda x: x + n) # n here is a free variable of the lambda, so the lambda is a closure. Therefore `adders` is a list of closures.
    return adders

adders = create_adders()

adders[0](10) # we get 14 instead of 11. 11 would come from when n = 1 in the loop and x = 10 so x+n = 11.

14

sidenote: We often hear "`lambdas` are just closures." But they are not; they are functions, and they only become closures once the `lambda` has a free variable.

We notice that `n` is mentioned in the loop's keyword but also within the `lambda`. Since `n` is not being passed as an argument, it's not within the scope of the `lambda`. Therefore, it is a free variable. There are now two mentions of `n` so a cell of 2 branches is created (`loop.n` and `lambda.n`) and points to the integer object `1`.

What happens when `n=2`? Well, **firstly**, Python goes through the `loop.n` branch and from there, it goes to the cell and makes it point to an integer object with value `2`. Then, when we reach the free variable `n` within the `lambda`, Python knows that it must point to the same `n` that `loop.n` points to. We established that `loop.n` is pointing to a cell that points to `2`, therefore `lambda.n` must now point to the same cell which points to `2`. Nothing is pointing to the integer `1` anymore! In fact, the cell will only point to the object created during the last iteration of the loop.

To summarise this is exactly the same as whats going on in the [example earlier on](#shared_extended_scope_example). The `count` in `inc1` and `inc2` both point to the same place as `count` in `outer`'s scope. They all point to a python cell. Here we have 4 different closures, all of which have an `lambda.n` which point to the same place as `create_adders.n` (local variable). So, these 5 references (4 `lambda.n` and 1 local variable `n`) all point to a python cell which initially points to an integer object with value `1`. Then, the loop statement makes `n = 2` so Python goes through the local branch and makes all 5 references now point to a new integer object of value `2`. And so on and so forth..

How do we fix this issue? The entire problem originates because, within the `lambda`, `n` is not being evaluated. Python recognises it as a free variable that's shared with the `n` in the loop statement. We need to therefore capture the value of `n` as the (`lambda`) function is being created. Here's the trick:

In [144]:
def create_adders():
    adders = []
    
    for n in range(1, 5):
        adders.append(lambda x, y=n: x + y) 
    
    return adders

adders = create_adders()

adders[0](10) # WORKS

11

Recalling from an earlier lecture, we learnt that if a default value is passed to a function, it is **evaluated during compile time, i.e. values created in memory**. Since `n` is no longer a free variable, we are **not** creating closures anymore, only functions. Here is a clear case showing that **`lambdas` are not necessarily closures**.

Here's a quick example of closures with disjointed extended scopes: 

In [145]:
def pow(n):
    # n is local to pow
    def inner(x):
        # x is local to inner
        return x ** n
    return inner

square = pow(2) # n = 2, and n is a free variable in the function inner, so inner (and hence square) has access to n = 2
cube = pow(3)

Both `square` and `cube` are closures with `n` as the free variable, albeit with different values - we have different cells for each of them and one cell points to `2` and the other to `3`. Note: numbers 2 and 3 have been interned by Python so they have preset memory addresses. Numbers from [-128, 127] are automatically interned.

In [146]:
print(square.__closure__, hex(id(2))) 
print(cube.__closure__, hex(id(3)))

(<cell at 0x7fb4f00fb130: int object at 0x7fb4f7718110>,) 0x7fb4f7718110
(<cell at 0x7fb4f00f9c30: int object at 0x7fb4f7718130>,) 0x7fb4f7718130


# 04 - Closure Applications - Part 1

In this example we are going to build an averager function that can average multiple values.

The twist is that we want to simply be able to feed numbers to that function and get a running average over time, not average a list which requires performing the same calculations (sum and count) over and over again. We could do this by a `class` approach, but we can see that it's not very efficient as we have to calculate `total` and `sum` with every added number.

In [147]:
class Averager:
    def __init__(self):
        self.numbers = []
    
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count
    
a = Averager()
print(a.add(10))
print(a.add(20))

10.0
15.0


How about with a closure? 

In [148]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers) 
        count = len(numbers)
        return total / count
    return add

a = averager() # a is now like add
print(a(10))
print(a(20))

10.0
15.0


The example above is still inefficient as we are summing and finding the length every time. Instead, we could just increment `total` and `count` with each added number. We saw that we were essentially able to convert a class to an equivalent functionality using closures. This is actually true in a much more general sense - very often, classes that define a single method (other than initializers) can be implemented using a closure instead.

In [149]:
def averager():
    total = 0
    count = 0
    
    def add(value):
        nonlocal total, count # this is essential because we have total and count in the lines below so Python will infer that its a local variable of add()
        total += value
        count += 1
        return 0 if count == 0 else total / count
    
    return add

a = Averager()
print(a.add(10))
print(a.add(20))

10.0
15.0


Now let's create a timer class: note that the dunder method `__call__` just tells Python that the instance that gets created from the class is a callable, and when it's called, it should execute this dunder method.

In [150]:
from time import perf_counter

class Timer:
    def __init__(self):
        self._start = perf_counter()
    
    def __call__(self):
        return (perf_counter() - self._start)
    
t1 = Timer()

In [151]:
t1() # wait 5 seconds before executing...

0.23788142400007928

In [152]:
t1() # ...this line

0.522584586999983

Equivalently, as a closure, we now have the same syntax as the class method for calling the function: (note that the name of the closure `elapsed` is not important because we never refer to it, we only return it.

In [153]:
def timer():
    start = perf_counter()
    
    def elapsed():
        # we don't even need to make start nonlocal 
        # since we are only reading it
        return perf_counter() - start
    
    return elapsed

t2 = timer()

In [154]:
t2()

0.2775421530004678

In [155]:
t2()

0.5290456709999489

# 05 - Closure Applications - Part 2

**Example 1:** Here we see a simple counter using closures, similar to the one we saw a few subsections ago, except we don't need to nest closures. This is the easier way of doing it. 

In [156]:
def counter(initial_value=0):
    
    def inc(increment):
        nonlocal initial_value
        initial_value += increment
        return initial_value
    
    return inc

counter1 = counter(10)
print(counter1(5))
print(counter1(5))

15
20


**Example 2:** Here we want to count the number of times a function is run using a **closure** 

In [157]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs) # fn is implicitly nonlocal because we reference it without there being any local assignment.
    
    return inner

def add(a, b):
    return a + b

counter_add = counter(add) #counter_add is basically equivalent to inner, so we can chuck in our function parameters and it will return the execute of the function

returned_value = counter_add(3, 4)
print(returned_value)

add has been called 1 times
7


We can improve on this by storing the number of counts in a dictionary for each function that's passed.

In [158]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)
    
    return inner

func_counters_dict = {}

def add(a, b):
    return a + b

def mult(a, b):
    return a*b

counted_add = counter(add, func_counters_dict)
counted_mult = counter(mult, func_counters_dict)

In [159]:
counted_add(3,4)
counted_add(5,6) 
counted_add(1,10) # called 3 times in total

counted_mult(5,6) # callled 1 time in total

func_counters_dict

{'add': 3, 'mult': 1}

But these `counted_add` and `counted_mult` variable names are awkward. Wouldn't it be good if we could use the name of the actual function? So, if we call `add`, it will execute `add` but also increment the dictionary value? All we do is simply rename `counted_add` to `add`, thereby overwriting `add` from a function to a closure. We'll demonstrate with a factorial function

In [160]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

fact = counter(fact, func_counters_dict)

print(fact(3))
print(fact(4))
print(fact(5))

func_counters_dict

6
24
120


{'add': 3, 'mult': 1, 'fact': 3}

So, it sounds like we're extending the functionality of a function. This leads us directly to decorators.

# 06 - Decorators - Part 1

In the last section, we were essentially making a decorator. We **wrapped** our `add` function with another function. We can also say that we **decorated** our `add` function with the `counter` function. We call `counter(fn)` a **decorator** function.

`add = counter(add)` \
`<closure> = <decorator>(<function to be decorated>)`

In general, for a **decorator** function:
- takes a function as an argument 
- returns a closure (usually).
- the closure usually accepts any (combo of) parameters via `(*args, **kwargs)`
- runs some code within the closure (e.g. increments a count, prints a statement etc.)
- the closure (which we've usually called inner) returns an execute of the function.

<img src=s7-images/7.9.png width=400/>

**The @ symbol**: It is purely for convenience.

Instead of:

In [161]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs) # fn is implicitly nonlocal because we reference it without there being any local assignment.
    
    return inner

In [162]:
def add(a, b):
    return a + b

add = counter(add)

We could do instead:

In [163]:
@counter
def add(a, b):
    return a + b

Remember what the decorator is doing. It really is just putting the function `add` as a parameter of `counter(fn)`

One big issue of this when debugging is that any introspection on the function `add` will infact point to `inner(*args, **kwargs)`. So, we've lost the name, docstring, parameter annotations etc.

In [164]:
print(add.__name__)
help(add)

inner
Help on function inner in module __main__:

inner(*args, **kwargs)



We can use a special function in the **functools** module, called **wraps**. 

In [165]:
from functools import wraps

In [166]:
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"{fn.__name__} was called {count} times")
    
    inner = wraps(fn)(inner)
    return inner

In fact, **wraps** is a decorator itself! 

Let's examine `inner = wraps(fn)(inner)`. Remember that we said decorators take a function as an argument which it decorates? Before, the `counter` function decorated the `add` function which gave `add` some extra functionality. 

Here `wraps`, the decorator, decorates the closure (inner) function (and it's going to collect the metadata that we want). As we also said earlier, it runs some code within the closure, e.g. incrementing a count. But, before it can do that, it needs to what the original function was. Indeed we could've decorated `add`, `multiply` etc., each with their own unique docstrings and parameter annotations.

So, we need to first pass this original function as an **argument** to the decorator, i.e., **decorator's can have parameters** -> `wraps(fn)`. \
Now we can go back and use this decorator to decorate `inner` -> `inner = wraps(fn)(inner)`.

But the above is written in 'closure notation'. We can use the convenient decorator notation of `@`:

```
def some_decorator(fn):

    def inner(*args, **kwargs):
        #code

    inner = wraps(fn)(inner)` 
```
simplifies to...

```
def some_decorator(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        #code
```

In [167]:
from functools import wraps

def counter(fn):
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))

    return inner

@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [168]:
help(add)

Help on function add in module __main__:

add(a: int, b: int = 10) -> int
    returns sum of two integers



To summarise, `inner` had the main functionality but it was missing other functionality such as accessing the metadata. We fixed that by decorating `inner` using the decorator `wraps`. This decorator requires the function whose metadata we want e.g. `add`'s metadata.

# 07 - Decorator Application - Timer

In the full notes, he creates 3 fibonacci generators, via recursion, a loop, and python's `reduce` - and times each one. 

First we will create our timer decorator. The `args_` and `kwargs_` are purely used for nice formatting; it creates a string that looks like the function's parameters, i.e., comma-separated. 

We use a 1-based system, e.g. first Fibonnaci number has index 1, etc.

In [169]:
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        print('{0}({1}) took {2:.6f}s to run.'.format(fn.__name__, 
                                                         args_str,
                                                         elapsed))
        return result
    
    return inner

**1. Recursion**

In [170]:
def calc_recursive_fib(n):
    if n <=2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)

In [171]:
@timed
def fib_recursed(n):
    return calc_recursive_fib(n)

In [172]:
fib_recursed(6)

fib_recursed(6) took 0.000017s to run.


8

Note that we did not decorate our recursive function (`calc_recursive_fib(n)`) directly. This is because, for each function call, it will recall the timer. See below..

In [173]:
@timed
def fib_recursed_2(n):
    if n <=2:
        return 1
    else:
        return fib_recursed_2(n-1) + fib_recursed_2(n-2)

fib_recursed_2(6)

fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(1) took 0.000064s to run.
fib_recursed_2(3) took 0.003015s to run.
fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(4) took 0.003032s to run.
fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(1) took 0.000000s to run.
fib_recursed_2(3) took 0.000015s to run.
fib_recursed_2(5) took 0.003061s to run.
fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(1) took 0.000000s to run.
fib_recursed_2(3) took 0.000014s to run.
fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(4) took 0.000029s to run.
fib_recursed_2(6) took 0.003105s to run.


8

Notice how inefficient this is. It has to calculate the same value multiple times even though it had calculated earlier. We can get around this using memoisation (caching).

**2. Loop**

In [174]:
@timed
def fib_loop(n):
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2     

fib_loop(7)

fib_loop(7) took 0.000002s to run.


13

**3. Using `reduce`**

We first need to understand how we are going to calculate the Fibonnaci sequence using reduce: 

<pre>
n=1:
(1, 0) --> (1, 1)

n=2:
(1, 0) --> (1, 1) --> (1 + 1, 1) = (2, 1)  : result = 2 

n=3
(1, 0) --> (1, 1) --> (2, 1) --> (2+1, 2) = (3, 2)  : result = 3

n=4
(1, 0) --> (1, 1) --> (2, 1) --> (3, 2) --> (5, 3)  : result = 5
</pre>

In general each step in the reduction is as follows:

<pre>
previous value = (a, b)
new value = (a+b, a)
</pre>

If we start our reduction with an initial value of `(1, 0)`, we need to run our "loop" n times.

We therefore use a "dummy" sequence of length `n` to create `n` steps in our reduce.

#### Quick aside on `reduce` from the 'realpython' website (skip if wanted)

In `reduce` we take 2 parameters (plus one extra for an initial value that jumps to the front of the iterable); what we do is: 
1. Apply a function (or callable) to the first two items in an iterable and generate a partial result.
2. Use that partial result, together with the third item in the iterable, to generate another partial result.
3. Repeat the process until the iterable is exhausted and then return a single cumulative value.

The first argument to Python’s reduce() is a two-argument function conveniently called function. This function will be applied to the items in an iterable to cumulatively compute a final value.

The second required argument, iterable, will accept any Python iterable. Note: If you pass an iterator to Python’s reduce(), then the function will need to exhaust the iterator before you can get a final value. So, the iterator at hand won’t remain lazy.

The third argument to Python’s reduce(), called initializer, is optional. If you supply a value to initializer, then reduce() will feed it to the first call of the first argument of the two-argument function.

#### `reduce` continued.

<pre>
for `n=7`, dummy = iterator that looks like (0, 1, 2, 3, 4, 5)

arg1: prev      arg2: x      ->      lambda return statement      ->      evaluated (this is what's held as the partial result)
    
(1, 0)            0                (prev[0] + prev[1], prev[0])                                (1, 1) 
    
(1, 1)            1                (prev[0] + prev[1], prev[0])                                (2, 1)     
    
(2, 1)            2                (prev[0] + prev[1], prev[0])                                (3, 2) 

(3, 2)            3                (prev[0] + prev[1], prev[0])                                (5, 3) 

(5, 3)            4                (prev[0] + prev[1], prev[0])                                (8, 5) 

(8, 5)            5                (prev[0] + prev[1], prev[0])                                (13, 8)
</pre>

A quick note on 'evaluated' written above: The `lambda` which takes two arguments, *normally* computes some comparison/operation *between* them and returns a single value, e.g. the max of two values, product etc. But here, we only actually care about `arg1`; from `arg1` alone we can get generate the next tuple whose first value is the next number in the fibonacci series. So, what role does `arg2` play?

In the first pairwise comparison (iteration), `arg2` is the first term in the `dummy` sequence -> 0. Python expected to 'compare/evaluate' with these two args, but instead we trick Python a little. Python normally expects a list/iterator and it will successively fold/reduce a pair of adjacent values, producing a partial solution for each pair, **until** we exhaust the items in the list (or exhaust the iterator). Here, we only want one thing: the partial solution after `n` foldings because each folding corresponds to generating the next term in the fibonacci sequence. It is basically recursion. So how can we tactically exhaust the list/iterator?. By creating a dummy sequence of appropriate length, we can guarantee `n` foldings of a pair of values  and thus, we can generate `n` terms in the fibonacci sequence.

**The function taken by `reduce` (`fib_gen_func`) has access to each element in the sequence during each iteration but NEVER uses it. It is merely for counting the folds.**

<pre>
sequence to iterate: (    (1, 0)    ,    0    ,    1    ,    2    ,     3     ,    4    ,    5    )

1st fold                    ^ -----------^                                                      
fib_seq                     (    (1, 1)    ,    1    ,    2    ,     3     ,    4    ,    5    )

2nd fold                            ^-----------^
fib_seq                            (    (2, 1)    ,    2    ,     3     ,    4    ,    5    )

3rd fold                                   ^-----------^
fib_seq                                  (    (3, 2)    ,     3     ,    4    ,    5    )

4th fold                                         ^------------^
fib_seq                                          (    (5, 3)    ,    4    ,    5    )

5th fold                                                 ^-----------^
fib_seq                                                (    (8, 5)    ,    5    )

6th fold                                                       ^-----------^
fib_seq                                                      (    (13, 8)    )

</pre>
So we were able to guarantee `n` terms in the fibonacci sequence by having a sequence of length `n-1`. (the initialiser `(1, 0)` included will make the sequence of length `n`.)

In [175]:
from functools import reduce

def fib_gen_func(prev, next_item_in_dummy):
    print(f"prev is {prev}, next item in dummy is {next_item_in_dummy}") # just to show you the adjacent values that are being 'used' (0,1,2.. etc. aren't used).
    return (prev[0] + prev[1], prev[0])

@timed
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n-1)
    fib_n = reduce(fib_gen_func, dummy, initial)
                                
    return fib_n[0]  

fib_reduce(7)

prev is (1, 0), next item in dummy is 0
prev is (1, 1), next item in dummy is 1
prev is (2, 1), next item in dummy is 2
prev is (3, 2), next item in dummy is 3
prev is (5, 3), next item in dummy is 4
prev is (8, 5), next item in dummy is 5
fib_reduce(7) took 0.000567s to run.


13

# 08 - Decorator Application - Logger, Stacked

In this example we're going to create a utility decorator that will log function calls (to the console, but in practice you would be writing your logs to a file (e.g. using Python's built-in logger), or to a database, etc.

Now we may additionally also want to time the function. We can certainly include the code to do so in our `logged` decorator, but we could also just use the `@timed` decorator we already wrote by **stacking** our decorators.

In [176]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner

def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [177]:
@logged
@timed
def fact(n):
    from functools import reduce
    from operator import mul
    return 0 if n < 1 else reduce(mul, range(1, n+1)) 

fact(5)

fact ran for 0.000005s
fact: called 2022-10-26 17:48:21.854223+00:00


120

This is identical to:

In [178]:
fact = logged(timed(fact))

As we can see, `timed(fact)` has to finish running (and printing) before `logged(timed(fact))` can finish running and printing. So, it's pretty intuitive. If we have moved the print statement to before the function call within the decorator, we would've seen the opposite order of print statements.

# 09 - Decorator Application - Memoization 

Recall how inefficient using recursion was for calculating the fibonacci sequence due to recalculating the same values multiple times.

In [179]:
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

We can fix this by caching calculated values into a dictionary. This concept of improving the efficiency of our code by caching pre-calculated values so they do not need to be re-calcualted every time, is called "memoization". We can write this as a class where the `cache` is an attribute. 

In [180]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}
    
    def fib(self, n):
        if n not in self.cache:
            print('Calculating fib({0})'.format(n))
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]
    
f_class = Fib()

In [181]:
f_class.fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


5

In [182]:
f_class.fib(5) 

5

Since, all values were in the cache, we didn't need to calculate any values. We just had to go to the cache. As we said earlier, it's usually possible to rewrite a simple class as a closure: It works exactly the same. 

In [183]:
def fib():
    cache = {1: 1, 2: 2}
    
    def calc_fib(n):
        if n not in cache:
            print('Calculating fib({0})'.format(n))
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    
    return calc_fib

f_closure = fib()

f_closure(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


8

Here we'll see how decorators and closures can be used differently. In particular, the generality of decorators. Let's rewrite the above as a decorator. This decorator will call a function with a particular argument e.g. `fib(3)` but only do so if a cache does not contain that value.

In [184]:
def memoize(fn):
    from functools import wraps
    cache = dict()
    print(f"cache has id: {hex(id(cache))}")
    
    @wraps(fn)
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args) # here we call any general function, but only if 'args' are not found in the cache.
        return cache[args]
    
    return inner

In [185]:
@memoize
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

cache has id: 0x7fb4da746200


In [186]:
@memoize
def fact(n):
    print('Calculating {0}!'.format(n))
    return 1 if n < 2 else n * fact(n-1)

cache has id: 0x7fb4f0304ec0


In [187]:
fib(3)

Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


2

In [188]:
fact(3)

Calculating 3!
Calculating 2!
Calculating 1!


6

**Important**: Recogise that, although both have the same `args` being passed to inner, the caches are different. And we see why above when we first decorated the function. Remember, the @ is purely convenient. It represents: `fib = memoize(fib)`. Therefore, even though it only seems to be a function definition, something is actually getting called. Since the print of the ID's ran, it shows us that the dictionaries were created prior to that moment.

Once we call `fib(3)`, it is like we're calling `inner`, **EXCEPT `fn` and `cache` are nonlocal variables (implicitly)** 

<img src=s7-images/7.10.png width=700 />

Our simple memoizer has a drawback however:
* the cache size is unbounded - probably not a good thing! In general we want to limit the cache to a certain number of entries, balancing computational efficiency vs memory utilization.
* we are not handling **kwargs

Memoization is such a common thing to do that Python actually has a memoization decorator built for us!

It's in the, you guessed it, **functools** module, and is called **lru_cache** and is going to be quite a bit more efficient compared to the rudimentary memoization example we did above.

[LRU Cache = Least Recently Used caching: since the cache is not unlimited, at some point cached entries need to be discarded, and the least recently used entries are discarded first]

In [189]:
from functools import lru_cache

@lru_cache(maxsize=4)
def fib(n):
    print("Calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

fib(4) 
print('')
fib(6)
print('')
fib(1) # As soon as fib(6) and fib(5) were determined and cached, Python deleted the very first ones: fib(1), fib(2), so we had to recalculate it if we call fib(1).

Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)

Calculating fib(6)
Calculating fib(5)

Calculating fib(1)


1

# 10 - Decorators - Part 2 

As we saw in the last subsection, it appeared as if we had decorators with parameters e.g. `@lru_cache(maxsize=16)`, so it seems we have a function call. This is in contrast to e.g. `@timed` where there is no function call.

In fact, that's exactly the case; only, this function call **returns** a decorator: `@lru_cache(max_size=16)` -> `@some_decorator`.

Consider the timed function with a hardcoded value of 10 which represents the number of cycles we average over. We can sort of fix the problem by accepting the parameter in the outer function, but this prevents us from using the `@` notation.
<pre>                                                                         bad approach</pre>
<img src=s7-images/7.11.png width=500 />  <img src=s7-images/7.12.png width=400 />

So, `timed` was a decorator that returned the `inner` closure. For it to remain a decorator, **`timed` must only take 1 parameter which is the function to be decorated**. The solution therefore is to have an even outer function that takes whatever parameters we want **and returns a decorator** 

`outer(reps=10)` ----returns----> `timed`

So, instead of `my_func = timed(my_func, reps=10)` we have `my_func = outer(reps=10)(my_func)` which is almost equivalent to `my_func = timed(my_func)`. The key thing to remember is that `timed` has access to `reps` because `reps` is a free variable. 

<img src=s7-images/7.13.png width=500 />

<a id='decorator_factory_image'></a>

**So, `outer` function is not itself a decorator. Instead, it returns a decorator, therefore it is a *decorator factory***

<img src=s7-images/7.14.png width=500 />

We don't really want to call things `factories`, so we instead have to cascade the names of all variables down once.

In [190]:
from functools import wraps

def timed(num_reps=1):
    def decorator(fn):
        from time import perf_counter

        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / num_reps
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                            num_reps))
            return result
        return inner
    return decorator  

In [191]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

def fib(n):      # We can't decorate `calc_fib_recurse(n)` because each recurse will recall the decorator. Instead we enclose it in a function and decorate that                  
    return calc_fib_recurse(n)

In [192]:
@timed(3)
def fib(n):
    return calc_fib_recurse(n)

In [193]:
fib(15)

Avg Run time: 0.000217s (3 reps)


610

In [194]:
@timed(100)
def fib(n):
    return calc_fib_recurse(n)

In [195]:
fib(15)

Avg Run time: 0.000250s (100 reps)


610

# 11 - Decorator Application - Decorator Class

Classes are things that allow you to have both state and functionality (properties and methods).

For relatively simple class structures you could use a closure (a functional approach) - that can have both state and functionality. But once the state/functionality becomes more complex, trying to do that using a functional approach (with closures) leads to more convoluted code - so the class approach is clearer - clarity of code is much preferred - writing complicated code that no one understands is usually not a good thing, so you can use either approach but one will usually be simpler/clearer.

Same thing with decorators - functional vs OOP. In most instances the functional approach is just fine, but you may get into situations where you need more complex functionality or state from your decorator, in which case the functional approach might just become too unwieldy and lack clarity.

Ultimately, you know how to do it either way, and you use the approach that bests suits the particular problem you are trying to solve. 

We'll create a decorator the functional way and a object-orientated way, then we will draw parallels between them. 

In [196]:
def my_dec(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print('decorated function called: a={0}, b={1}'.format(a, b))
            return fn(*args, **kwargs)
        return inner
    return dec

@my_dec(10, 20)
def my_func(s):
    print('hello {0}'.format(s))
    
my_func('world')

decorated function called: a=10, b=20
hello world


So, our decorator factory was passed some arguments, and returned a callable which took one single parameter, the function being decorated, but also had access to the arguments passed to the factory. 

With classes we normally have `obj = MyClass(10, 20)`. The RHS returns an object, but we can make it a callable by using the `__call__` --> `obj(10, 20)` works. 

So, `my_dec(a, b)`, the decorator factory, returned a decorator called `dec` that takes **only one single param, the func being decorated** -> `dec(fn)`.\
Cf. `MyClass(10, 20)` (decorator factory class), returns an obj. It's called by `__call__` (the decorator) that takes **only one single param, the function being decorated** -> `__call__(self,fn)`.

**The object *is* a decorator because it can be called with the function that we want to decorate.**

In [197]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, fn):
        def inner(*args, **kwargs):
            print('MyClass instance called: a={0}, b={1}'.format(self.a, self.b))
            return fn(*args, **kwargs)
        return inner
    
@MyClass(10, 20)
def my_func(s):
    print('Hello {0}!'.format(s))
    
my_func('Python')

MyClass instance called: a=10, b=20
Hello Python!


The parallels therefore...
<pre> 
Decorator Factory:            my_dec(a,b)                 ==    MyClass(a,b)     
Decorator:                    dec(fn)                     ==    __call__(fn, self)
Decorator parameters a, b:    taken directly from my_dec  ==    taken via __init__(self, a, b) 
</pre>

# 12 - Decorator Application (Decorating Classes)

We have so far worked with decorating functions. This means we can decorate functions defined with a `def` statement (we can use the `@` syntax, or the long form). Since class methods are functions, they can be decorated too. Lambda expressions can also be decorated (using the long form).

But if you think about how our decorators work, they take a single parameter, a function, and return some other function - usually a closure that uses the original function that was passed as an argument.

We could use the same concept to accept, not a function, but a class instead. We could reference that class inside our decorator, modify it, and then return that modified class.

First we look at something called **monkey patching**. It boils down to modifying or extending our code at **run time**.

For example we can modify or add attributes to classes at run time. Modules too.

In Python, many of the classes we use can be modified at run time 
(built-ins like strings, lists, and so on, cannot).

But classes written in Python, such as the ones we write, and even library classes, as long as they are written in Python, not C, can. For example `Fraction` in the `fractions` module can be monkey patched.

Just because we can do something however, does not mean we should! Monkey patching can be extremely useful, but don't do it just because you can - as always there should be a real reason to do it, as we'll see in a bit.

Also, in general it is a bad idea to monkey patch the special methods `__???__` (such as `__len__`) as this will often not work due to how these methods are searched for by Python.

If we instantiate an instance of the Fraction class..

In [198]:
from fractions import Fraction

f = Fraction(2,3)

and then add a new attribute to the Fraction Class with:

In [199]:
Fraction.speak = 100

We are then able to access that attribute from your previously instantiated instance. So, all instances have access to the class's attributes even if they were instantiated before those attributes were added to the class? In other words, python looks for speak on `f` and failing to find it there finds it as a class attribute and uses that one.

In [200]:
f.speak

100

We can add methods instead of just attributes but be sure to remember the `self` parameter. Once ran, this new method will be available to all instances of the `Fraction` Class.

In [201]:
Fraction.speak = lambda self, message: f'Fraction says {message}.'
f.speak('This is a late a parrot')

'Fraction says This is a late a parrot.'

In [202]:
f2 = Fraction(10, 5)
f2.speak('This parrot is no more')

'Fraction says This parrot is no more.'

This is **monkey patching**. If our Fraction class lacks some functionality that we want, we can **monkey patch** the Fraction class and add in the new method.

If you want a more useful method, how about one that tells us if the Fraction is an integral number? (i.e. denominator is `1`)

In [203]:
Fraction.is_integral = lambda self: self.denominator == 1

In [204]:
f1 = Fraction(1, 2)
f2 = Fraction(10, 5)

print(f1.is_integral())
print(f2.is_integral())

False
True


Instead of writing it out explicitly, let's write it in a function that monkey patches any given class.

In [205]:
def dec_speak(cls):
    cls.speak = lambda self, message: f'{self.__class__.__name__} says {message}'
    return cls  # We don't really have to do this, 
                # because we've modified the object, just like when we append to a list: It does the thing and we don't care about the return of some_list.append()
 
Fraction = dec_speak(Fraction) #look familiar?
f.speak('This is a late parrot.') # For it to be identical to the last subsection, we would have Fraction.speak(), but here we have an instance of it.

'Fraction says This is a late parrot.'

Again, we don't have to write `Fraction = dec_speak(Fraction)` because `dec_speak` is performing a mutation. We could instead remove the return inside `dec_speak` and write `dec_speak(Fraction)` and that would achieve the same. This way was purely to demonstrate the similarity with decorators.

Now, what if we had another class and we wanted to `decorate` that with the ability to speak?

In [206]:
class Person:
    pass

Person = dec_speak(Person) # This is one way of decorating from last subsection.
p = Person()
p.speak('Hi')

'Person says Hi'

**Example Use Case:** As a first example, let's say you typically like to inspect various properties of an object for debugging purposes, maybe the memory address, it's current state (property values), and the time at which the debug info was generated.

In [207]:
from datetime import datetime, timezone

def debug_info(cls):
    def info(self):    # NOTE: This is not a closure; there are no free variables within this, thus, it could've been defined outside of debug_info(cls). 
                       #       but it's just so closely related. In fact, its better to keep it outside so we don't need to recreate it every single time.
        results = []
        results.append('time: {0}'.format(datetime.now(timezone.utc)))
        results.append('class: {0}'.format(self.__class__.__name__))
        results.append('id: {0}'.format(hex(id(self))))
        
        if vars(self):
            for k, v in vars(self).items(): #vars(self) simply returns all the attributes/properties that an object has, as a key-value pair.
                results.append(f'{k}: {v}')
        
        # we have not covered lists, the extend method and generators,
        # but note that a more Pythonic way to do this would be:
        # if vars(self):
        #    results.extend(f"{k}: {v}" for k, v in vars(self).items())
        #                   
        
        return results
    
    cls.debug = info
    
    return cls # this line technically isn't necessary cos we're mutating a class. 
               # But it will be necessary if we want to use the `Person=debug_info(Person)` or even the `@debug_info` approach because they are both identical.
               # We only don't need it if we write `debug_info(Person)` WITH NO ASSIGNMENT. Otherwise since `debug_info(Person)` returns None, the LHS of
               # 'Person' will be equated to the None object. So, when we do p = Person('John'), we are effectively doing None('John') on the RHS. 

A quick note: Last time, we wrote a method to our class using a lambda: `cls.speak = lambda self, message:`. Since we have something more complicated this time, we need to use a function: `def info(self):`. We must remember to give `self` because when we create a method manually, we write: `def some_method(self, other_params):`. This `self` will take the object that called it. If you can't remember this from earlier, just remember that all methods need to take `self` as their first parameter, and that it's easiest to think of `def info(self):` as a method as if you were directly writing it into your own class. .

In [208]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        
    def say_hi():
        return 'Hello there!'
    
p1 = Person('John', 1939)

In [209]:
p1 = Person('John', 1939)
p1.debug()

['time: 2022-10-26 17:48:37.108789+00:00',
 'class: Person',
 'id: 0x7fb4f00fb310',
 'name: John',
 'birth_year: 1939']

This is a good use of decorators because we can reuse it on many classes.

Let's look at another example. [Here](../Section%2002%20-%20A%20Quick%20Refresher/Section%202%20Summary.ipynb) is where you can find a refresher on getter/setter decorators. In short, adding `@property` to some method `def value(self)` which returns `self.value` means that when users seemingly call an attribute `obj.value`, it is infact looking for the `value` method which returns the `self.value` attribute. This makes the class more dynamic, and it's a good habit to get all attributes that may change using a getter. The reason why is because if we have an attribute that is made from other attributes, then this attribute won't get updated if the other attributes are updated because all attributes are initialised only once.

A setter is the proper way of updating an attribute. If we have an attribute called `value` and we want to update it using `obj.value = new_value`, we should instead create a method called `def value(self, new_value)` with a decorator: `@value.setter` which updates the attribute via `self.value = new_value`.

In [210]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed_mph):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed_mph = top_speed_mph
        self.current_speed = 0
    
    @property
    def speed(self):
        return self.current_speed
    
    @speed.setter
    def speed(self, new_speed):
        self.current_speed = new_speed

In [211]:
car1 = Automobile('Ford', 'Model T', 1908, 45)
car1.debug()

['time: 2022-10-26 17:48:37.906237+00:00',
 'class: Automobile',
 'id: 0x7fb4f00f9270',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 0']

In the full notes there's an example which shows that if you have a class which has `__eq__` and `__lt__` implemented, then you can build all the other ordering operators, such as 'not equal', 'greater than or equal to' etc., using logical operators. It should be done using a decorator which takes a class as a parameter and then monkeypatches these additional methods. This is a better approach than implementing all dunder methods (`__neq__`, `__ge__`) within the class itself.

But then, as is common with Python, there's already a decorator implementation for this exact purpose found in the standard library. It's imported using `from functools import total_ordering` and we just decorate our class using `@total_ordering`. It is more versatile than our implementation because, all it requires is `__eq__` and only one other ordering dunder e.g. `__ge__`, and the rest will be monkeypatched. 

# 13 - Decorator Application - Single Dispatch

Read the full notes for this section. The concepts are useful but because Python will have its own implementation for doing this, it's easier to just skim through the appraoch until you reach the correct way. I have pasted the intro from the full notes below...

<pre>

Consider an application where we want to provide similar functionality but that varies slightly depending on the argument types passed in.

In this set of examples we consider this problem where functionality differs based on a single argument's type (hence single dispatch) instead of the type of multiple arguments (which would be multi dispatch).

If you have a background in some other OO languages such as Java or C#, you'll know that we can easily do something like this by basically **overloading** functions: using a different data type for the function parameter, hence changing the function signature. Then although the name of the function is the same, calling `do_something(100)` and `do_something('java')` would call a different function, the first one would call the `do_something(int)` function, and the second would call the `do_something(String)` function.

Of course, Python is not statically typed, so even if Python had function overloading built-in, we would not be able to make such a distinction in our function signatures since there is nothing that says that a parameter must be of a specific type, so in a best case scenario we would have to "distinguish" functions with the same name only by the number of parameters they take. And then we'd have to somehow deal with variable numbers of positional and keyword arguments too... Uuugh!
In any event, single dispatch could never work.

Instead we have to come up with a different solution.

Let's say we want to display various data types in html format, with different presentations for integers (we want both base 10 and hex values), floats (we always want it rounded to 2 decimal points), strings (we want the string html-escaped, and all newline characters replaced by `<br/>`), lists and tuples should be implemented using bulleted lists, and the same with dictionaries except we want the name/value pair to be displayed in the bulleted list.

</pre>


A single dispatcher is a function that takes in an argument, and based on the type of argument, it calls a specific function. For example, in this section we have `htmlize(arg)` which takes any input and converts it into something that can be understood by HTML; lists for example need `<li> </li>` to enclose each list item, while regular strings containing characters like `>` need to be HTML escaped. Depending on what the argument is and its type, we will need to treat it differently within the function. Here's the example below.

To quickly remind what's going on below: We are creating a decorator which, when the `@singledispatch` line is reached, will **immediately run, create the extended scope (containing the registry dict), and return a callable (func without the brackets)**. From Python's perspective, we have no knowledge of `htmlize`'s local variable called `a`. Normally we have the function execute within `inner`. Here, we still do, except we store the function in a dictionary and then call it from the dictionary.

In [212]:
def singledispatch(fn):
    print('decorator executed and registry created')
    registry = dict()
    registry[object] = fn
    registry[int] = lambda arg: '{0}(<i>{1}</i)'.format(arg, str(hex(arg)))
    registry[float] = lambda arg: '{0:.2f}'.format(round(arg, 2))
    
    def inner(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)
    return inner

Remember, we could get rid of `@singledispatch` and add `htmlize = singledispatch(htmlize)` below the `htmlize(a)` function and get the exact same functionality.

In [213]:
@singledispatch
def htmlize(a):
    return escape(str(a))

decorator executed and registry created


Now, we want a way to add the specialized functions to the `registry` dictionary from **outside** the `singledispatch` function; right now, `singledispatch` is not generic as we had to hardcode the registry entries and the lambda functions are specific to this `htmlize` function. We want to be able to use this decorator `singledispatch` on any function. To do so we will create a parametrized decorator that will (1) take the type as a parameter, and (2) return a closure that will decorate the function associated with the type.

We're going to create a **decorator factory** within our `singledispatch` decorator.

See [these cells in Decorators - Part 2](#decorator_factory_image) to see that a decorator factory has a decorator function defined within it, which has an inner defined within that. Inner executes the function and returns it  (or returns the function execute itself) to the `decorator`. Then, the `decorator` returns `inner` (a callable, ready to be executed) to the `decorator factory`. Then the `decorator factory` returns the `decorator` (a callable, ready to decorate). So, when the user calls the decorator factory with a decorator as a parameter, it returns a decorator (callable, ready to decorate). When the user applies the decorator to some function, it returns a closure called inner (callable, ready to execute). Finally, when the user calls the function, it executes `inner` which will call the function somewhere within it.

But here we don't have that! We have the factory `register(type_)` and the decorator called `inner(fn)` below, but we don't have the closure within `inner`. Firstly, let's remind you that `fn`, the parameter of `inner`, is the function to be decorated. All we do within the decorator is assign the function to the type in the `registry` dictionary.

In [214]:
def singledispatch(fn):
    registry = dict()
    
    registry[object] = fn
    
    def register(type_): # this is a decorator factory. It will allow us to create parametrised decorators. The parameter is `type_`
        def inner(fn):
            registry[type_] = fn # registry is nonlocal. It is defined in the outer `singledispatch` so its a free variable within the 'register' closure.
        return inner
        
    
    def decorated(arg): # this is our 'inner'
        fn = registry.get(type(arg), registry[object])
        return fn(arg)
    
    return decorated

But of course this is not good enough - how do we get a hold of the `register` function from outside `singledispatch`? Remember, `singledispatch` is a decorator that returns the `decorated` closure, not the `register` closure.

We can do this by adding the `register` function as an **attribute** of the `decorated` function before we return it. 

While we're at it we're also going to:

* add the `registry` dictionary as an attribute as so we can look into it to see what it contains.

* add another function that given a type will return the function associated with that type (or the default function if the type is not found in the dictionary)

In [215]:
def singledispatch(fn):
    registry = dict()
    
    registry[object] = fn
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn  # we do this so we can stack register decorators!
        return inner
   
    def decorated(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)
    
    def dispatch(type_):
        return registry.get(type_, registry[object])

    decorated.register = register
    decorated.registry = registry.keys()
    decorated.dispatch = dispatch
    return decorated

I will first prove that the `htmlize` is infact equivalent (or points) to `decorated`. Therefore, if `decorated` has attributes then we can access them via `htmlize`.

In [216]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [217]:
htmlize

<function __main__.singledispatch.<locals>.decorated(arg)>

Now we can run our decorator with parameters (i.e. our decorator factory) on a function that defines one of those lambda functions we saw above, e.g. enclose integers with `<i></i>`.

In [218]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

To summarise, `htmlize.register` gives us access to the `register decorator factory`. 

Once we put in a parameter e.g. `int`, it executes the decorator factory and produces a decorator.

`htmlize.register(int)` is basically equivalent to `inner`. Because it's a decorator, once we add the `@` in front of it, it executes `inner` passing in the function below the `@`. Thus, `inner(fn)` becomes `inner(html_int)`. (Note that since `inner(fn)` returns `fn`, thus `inner(html_int)` returns `html_int`.)

This execution (`@htmlize.register(int)`) produces a key called `int` with a value called `html_int` - this is a callable (there are no brackets on it). Normally, we would execute `html_int(<some integer>)` somewhere within the global scope. But in this case, it is executed if `htmlize` receives an integer because `htmlize` will look for the `int` key and then apply `html_int` with the given integer.

Here's why this approach is very versatile: The cell above returns `html_int(a)`; that's what decorators do - they add some functionality but ultimately return the function back (remember the long form: `html_int = htmlize.register(int)(html_int)`). This means that we can stack other decorators e.g. `@htmlize.register(tuple)` on top of `@htmlize.register(int)`  and it will first decorate it with `@htmlize.register(int)`, returning the original function so that it can be decorated again by `@htmlize.register(tuple)`. Both keys `tuple` and `int` will then have the same function as their value.

In [219]:
html_int

<function __main__.html_int(a)>

In [220]:
htmlize(100)

'100(<i>0x64</i)'

Everything works. If we want to ask what function `htmlize` will use for integers, we just type:

In [221]:
htmlize.dispatch(int) # expect it to tell us the 'html_int' function.

<function __main__.html_int(a)>

Our single dispatch decorator works quite well - but it has some limitations. For example it cannot handle functions that take in more than one argument (in which case dispatching would be based on the type of the **first** argument), and we also are not allowing for types based on parent classes - for example, integers and booleans are both integral numbers - i.e. they both inherit from the Integral base class. Similarly lists and tuples are both more generic Sequence types. We'll see this in more detail when we get to the topic of abstract base classes (ABC's).

We can use `isinstance(some obj from some class, some class)`

In [222]:
class Person:
    pass

class Student(Person):
    pass

p = Student()

print(f'Type is: type(p)')
print(f'Does p inherit from Person? {isinstance(p, Person)}')

Type is: type(p)
Does p inherit from Person? True


The solution is to use abstract base classes but we'll see that in Part 4.

As mentioned at the start of this section, Python already has an implementation for singledispatch which is imported using `from functools import singledispatch`. It is a decorator that decorates some function, which we've called `htmlize`. Then, this decorated function accepts different types e.g. bool, int, str etc. It will then let us write a function for each of these types that will execute when this type is passed to `htmlize`. If we haven't written any functions, then it will default to `htmlize`.

It also applies an attribute called `dispatch` which lets us check which function is being used for which type, as well as a `registry` attribute which is the equivalent to our dictionary which stores these types (keys) with their functions (values).

In [223]:
from functools import singledispatch
from collections.abc import Sequence
from numbers import Integral
from html import escape

@singledispatch
def htmlize(a):
    return escape(str(a))

In [224]:
htmlize.dispatch(bool)

<function __main__.htmlize(a)>

Let's register a function to be used for Integral numbers (integers and bools). This wasn't working in our implementation due to inheritance issues.

In [225]:
@htmlize.register(Integral)
def html_integral_number(a):
    return f"{a}(<i>{str(hex(a))})</i>)"

htmlize.registry # Checking to see if the function has been added to our dictionary. Note 'object' is the default because everything in Python is an object.

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.html_integral_number(a)>})

In [226]:
print(htmlize.dispatch(int)) # Finding the function associated with integers.
print(htmlize.dispatch(bool)) # Finding the function associated with booleans.

<function html_integral_number at 0x7fb4da63c940>
<function html_integral_number at 0x7fb4da63c940>


Now let's register all sequence types (tuples, lists etc). We can see that it takes each item in the sequence and calls `htmlize` on them. This is so that if we have an item that's an integer and another item that's a bool, they are both treated with the function that we've set for that type.

In [227]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = [f'<li>{htmlize(item)}</li>' for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [228]:
htmlize([10, (3,4), True])

'<ul>\n<li>10(<i>0xa)</i>)</li>\n<li><ul>\n<li>3(<i>0x3)</i>)</li>\n<li>4(<i>0x4)</i>)</li>\n</ul></li>\n<li>True(<i>0x1)</i>)</li>\n</ul>'

It all worked. But there's a problem; strings are also sequence types because you can iterate through them. We can double check using `isinstance('helloworld', Sequence)` 

In [229]:
htmlize.dispatch(str)

<function __main__.html_sequence(l)>

In [230]:
htmlize('abc')

RecursionError: maximum recursion depth exceeded while calling a Python object

Let's see why this happened. Since `abc` is a string which is an instance of `Sequence`, we use the `html_sequence` function. In this function, we apply the function `htmlize(item)` for each item. The first item is 'a', so we call `htmlize('a')` which is a string which is an instance of `Sequence`, so we use the `html_sequence` function. There's only one item in the sequence but still we apply `htmlize(item)` on this item. So this recurses indefinitely.

All we have to do to fix this is create a function for strings.

In [231]:
@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n', '<br/>\n')

htmlize('python 1<100')

'python 1&lt;100'

So, even though a string is both a `str` instance and in general a sequence type, the "closest" type will be picked by the dispatcher (again something our own implementation did not do). By closest, we mean lower in the class hierarchy of inheritance. So 'python' is more specifically a string than a Sequence. 

This means, we have something for generic sequences, but something specific for more specialized strings.

Functions like `html_sequence`, `html_str` are not accessed directly. Instead, they are accessed through the `singledispatch`. So, the names of them are unimportant and infact, we'd prefer that other users know that they're not supposed to access them directly. So you may often see *all* of these functions renamed with `_` to indicate that this is (pseudo)private. Even if all these functions have the same name, they will still execute perfectly through `singledispatch` because Python doesn't refer to functions by their labels but what those labels point to in memory.

It's the same reason why we use `_` when unpacking a list but we only care about one or two of the values.

In [232]:
a, *_, c = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(a)
print(_)
print(c)

1
[2, 3, 4, 5, 6, 7, 8, 9]
10
